## Iris Flower classification


In [ ]:
from __future__ import absolute_import , division , print_function , unicode_literals

import tensorflow as tf
import pandas as pd
import numpy as np
# Check TensorFlow version
print(tf.__version__)

# Create a simple tensor
# tensor = tf.constant([[1, 2], [3, 4]])
# print(tensor)
# tensor_reshape = tf.reshape(tensor,[4,1])
# print(tensor_reshape)


2025-06-14 21:54:38.457994: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-14 21:54:38.558696: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-14 21:54:38.648942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749917378.728527   19307 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749917378.754912   19307 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749917378.934010   19307 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

2.19.0


In [2]:
CSV_COLUMN_NAMES = ['SepalLength' , 'SepalWidth' , 'PetalLength' , 'PetalWidth' , 'Species']
SPECIES = ['Setosa'  , 'Versicolor' , 'Virginica']
#Constants to help us later on

In [3]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv" , "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
)
test_path = tf.keras.utils.get_file(
    "iris_test.csv" , "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
)
# Load the datasets into pandas DataFrames
# Use header=0 to read the existing header, adjust if no header is present
train = pd.read_csv(train_path ,names =CSV_COLUMN_NAMES, header =0)
test = pd.read_csv(test_path ,names = CSV_COLUMN_NAMES, header = 0)
# Verify column names
print("Train columns:", train.columns.tolist())
print("Test columns:", test.columns.tolist())

Train columns: ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
Test columns: ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']


In [4]:
# Separate labels and convert to integer encoding if needed
train_y = train.pop('Species')
test_y = test.pop('Species')
# Convert features to numpy array
X_train = train.values
X_test = test.values

In [5]:
# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X_train)  # Compute mean and variance

E0000 00:00:1749917389.813368   19307 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1749917389.816475   19307 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Tensor Flow data set

In [6]:


def create_dataset(features, labels, training=True, batch_size=256, num_epochs=None):
    """
    Creates a TensorFlow Dataset from features and labels.
    
    Args:
        features (dict): Dictionary of feature tensors or numpy arrays.
        labels: Label tensor or numpy array.
        training (bool): Whether to shuffle and repeat the dataset (default: True).
        batch_size (int): Size of each batch (default: 256).
        num_epochs (int, optional): Number of epochs to repeat the dataset (default: None, infinite if training).
    
    Returns:
        tf.data.Dataset: A batched and optionally shuffled/repeated dataset.
    """
    # Convert features dictionary and labels to a Dataset
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    
    # Shuffle and repeat if in training mode
    if training:
        dataset = dataset.shuffle(buffer_size=1000)  # Shuffle with a buffer of 1000 samples
        if num_epochs is not None:
            dataset = dataset.repeat(num_epochs)  # Repeat for specified epochs
        else:
            dataset = dataset.repeat()  # Infinite repeat if no epochs specified
    
    # Batch the dataset
    dataset = dataset.batch(batch_size)
    
    # Prefetch to improve performance
    dataset = dataset.prefetch(tf.data.AUTOTUNE)  # Allows background processing of data
    
    return dataset

## Training and Testing Datasets

In [7]:
# Create datasets
train_dataset = create_dataset(X_train, train_y, training=True, batch_size=32, num_epochs=10)
test_dataset = create_dataset(X_test, test_y, training=False, batch_size=32, num_epochs=1)

In [8]:
# Build the Keras model for multiclass classification
model = tf.keras.Sequential([
    normalizer,  # Apply normalization layer
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 units for 3 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # For integer labels
              metrics=['accuracy'])



## Training and Evaluation

In [36]:
# Train the model
model.fit(train_dataset, epochs=10, steps_per_epoch=len(X_train) // 32)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset, steps=len(X_test) // 32)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9466 - loss: 0.1644
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9818 - loss: 0.1589
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9466 - loss: 0.1448
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9648 - loss: 0.1430
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9427 - loss: 0.1561
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9505 - loss: 0.1467
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9805 - loss: 0.1249
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9766 - loss: 0.1387
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9388 - loss: 0.1465
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9557 - loss: 0.1575
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.1669
Test Loss: 0.1669, Test Accuracy: 1.0000


## Predictions

In [ ]:
# Optional: Make predictions
pred_probs = model.predict(test_dataset, steps=len(X_test) // 32)
pred_classes = tf.argmax(pred_probs, axis=1).numpy()


#Using User Input to display Predictions
def user_input_prediction(user_input):
    user_input_array = np.array([user_input])
    user_input_normalized = normalizer(user_input_array)
    predictions = model.predict(user_input_normalized)
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]
    class_names = ['Setosa', 'Versicolor', 'Virginica']
    print(f"Predicted class: {class_names[predicted_class]}")

user_input = [
    float(input("Sepal length: ")),
    float(input("Sepal width: ")),
    float(input("Petal length: ")),
    float(input("Petal width: "))
]
user_input_prediction(user_input)

# Print some results
for i in range(min(10, len(test_y))):
    print(f"Sample {i+1}: Predicted = {pred_classes[i]}, Actual = {test_y.iloc[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Sample 1: Predicted = 1, Actual = 1
Sample 2: Predicted = 2, Actual = 2
Sample 3: Predicted = 0, Actual = 0
Sample 4: Predicted = 1, Actual = 1
Sample 5: Predicted = 1, Actual = 1
Sample 6: Predicted = 1, Actual = 1
Sample 7: Predicted = 0, Actual = 0
Sample 8: Predicted = 2, Actual = 2
Sample 9: Predicted = 1, Actual = 1
Sample 10: Predicted = 2, Actual = 2
